In [15]:
import pandas as pd
import numpy as np
import re

class FertilizerStandardizer:
    def __init__(self):
        # Garder les mêmes initialisations...
        self.icv_fertilizer_list = ["Solution Liquide N 39"]
        self.default_mineral_density = 1.325
        self.organic_solid_density = 1000
        self.organic_liquid_density = 1.0
        self.type_mapping = {
            "Fertilisation et amendement organique": "engrais organiques",
            "Fertilisation et amendement mineral - foliaire inclus": "engrais mineraux",
            "Fertilisation organique": "engrais organiques",
            "Fertilisation minérale": "engrais mineraux"
        }

    # Garder les méthodes auxiliaires identiques...
    def standardize_fertilizer_name(self, name):
        if pd.isna(name):
            return name
        name = str(name).upper()
        name = re.sub(r'[^\w\s]', ' ', name)
        name = re.sub(r'\s+', ' ', name)
        return name.strip()

    def is_in_icv_list(self, fertilizer_name):
        std_name = self.standardize_fertilizer_name(fertilizer_name)
        std_icv_list = [self.standardize_fertilizer_name(name) for name in self.icv_fertilizer_list]
        return std_name in std_icv_list

    def standardize_type_engrais(self, intervention_type):
        if pd.isna(intervention_type):
            return "engrais mineraux"
        type_std = self.type_mapping.get(intervention_type)
        return type_std if type_std else "engrais mineraux"

    def calculate_nutrients(self, row):
        """Calcule les teneurs en nutriments NPK pour les engrais non-ICV"""
        try:
            if not row['in_icv_list']:
                if row['type_engrais'] == 'engrais organiques':
                    # Pour les engrais organiques, les compositions sont en kg/m³
                    # On utilise l'hypothèse 1T = 1m³
                    volume_m3 = row['quantite_totale'] if row['unite_intrant_intervention'].upper() == 'M3' \
                        else row['quantite_kg'] / self.organic_solid_density
                    
                    row['N_kg_ha'] = row['fertilisant.composition.n_total'] * volume_m3 / row['surface_travaillee_ha']
                    row['P_kg_ha'] = row['fertilisant.composition.p'] * volume_m3 / row['surface_travaillee_ha']
                    row['K_kg_ha'] = row['fertilisant.composition.k'] * volume_m3 / row['surface_travaillee_ha']
                else:
                    # Pour les engrais minéraux, les compositions sont en pourcentage
                    row['N_kg_ha'] = row['dose_kg_ha'] * row['fertilisant.composition.n_total'] / 100
                    row['P_kg_ha'] = row['dose_kg_ha'] * row['fertilisant.composition.p'] / 100
                    row['K_kg_ha'] = row['dose_kg_ha'] * row['fertilisant.composition.k'] / 100
            return row
        except Exception as e:
            print(f"Erreur lors du calcul des nutriments pour l'engrais {row['libelle']}: {str(e)}")
            return row

    def convert_to_kg(self, row):
        # Garder la même logique de conversion...
        try:
            if pd.isna(row.get('unite_intrant_intervention')) or pd.isna(row.get('quantite_totale')):
                return np.nan
                
            unite = str(row.get('unite_intrant_intervention', '')).upper()
            type_engrais = str(row.get('type_engrais', '')).lower()
            quantite = float(row.get('quantite_totale', 0))
            
            if unite == 'KG':
                return quantite
            elif unite == 'L':
                return quantite * (self.organic_liquid_density if 'organique' in type_engrais else self.default_mineral_density)
            elif unite == 'M3':
                return quantite * (self.organic_solid_density if 'organique' in type_engrais else self.default_mineral_density * 1000)
            else:
                print(f"Avertissement: Unité non reconnue: {unite} pour l'engrais {row.get('libelle', 'Unknown')}")
                return np.nan
        except Exception as e:
            print(f"Erreur lors de la conversion pour l'engrais {row.get('libelle', 'Unknown')}: {str(e)}")
            return np.nan

    def standardize_mesparcelles(self, data):
        print("Standardisation des données MesParcelles...")
        
        # Filtrer les fertilisations
        fertilisation_types = [
            "Fertilisation et amendement organique",
            "Fertilisation et amendement mineral - foliaire inclus"
        ]
        df = data[data['type_intervention.libelle'].isin(fertilisation_types)].copy()
        print(f"Après filtrage : {len(df)} lignes")
        
        # Vérifier la présence des colonnes requises
        required_columns = [
            'libelle', 'unite_intrant_intervention', 'quantite_totale', 
            'surface_travaillee_ha', 'fertilisant.composition.n_total',
            'fertilisant.composition.p', 'fertilisant.composition.k',
            'type_intervention.libelle', 'siret_exploitation', 'uuid_parcelle'
        ]
        
        df = df.dropna(subset=required_columns)
        
        # Standardisation
        df['type_engrais'] = df['type_intervention.libelle'].apply(self.standardize_type_engrais)
        df['in_icv_list'] = df['libelle'].apply(self.is_in_icv_list)
        df['quantite_kg'] = df.apply(self.convert_to_kg, axis=1)
        df['dose_kg_ha'] = df['quantite_kg'] / df['surface_travaillee_ha']
        
        # Calcul des nutriments
        df = df.apply(self.calculate_nutrients, axis=1)
        
        return self._aggregate_by_exploitation(df, 'siret_exploitation')

    def standardize_smag(self, data):
        print("Standardisation des données SMAG...")
        
        # Filtrer les fertilisations
        fertilisation_types = ["Fertilisation organique", "Fertilisation minérale"]
        df = data[data["Type d'intervention"].isin(fertilisation_types)].copy()
        
        # Renommage des colonnes
        df = df.rename(columns={
            'Intrant': 'libelle',
            'Unité': 'unite',
            'Dose': 'dose',
            'Surface intervention sur parcelle': 'surface_travaillee_ha',
            'N': 'N_kg_ha',
            'P': 'P_kg_ha',
            'K': 'K_kg_ha',
            'SIRET': 'siret_exploitation',
            'Code edi parcelle': 'uuid_parcelle'
        })
        
        # Standardisation
        df['type_engrais'] = df["Type d'intervention"].apply(self.standardize_type_engrais)
        df['in_icv_list'] = df['libelle'].apply(self.is_in_icv_list)
        
        # Conversion des doses pour les engrais ICV
        df.loc[df['in_icv_list'], 'dose_kg_ha'] = df[df['in_icv_list']].apply(
            self.convert_dose_to_kg_ha, axis=1)
        
        return self._aggregate_by_exploitation(df, 'siret_exploitation')

    def convert_dose_to_kg_ha(self, row):
        """Convertit les doses SMAG en kg/ha"""
        try:
            if pd.isna(row.get('unite')) or pd.isna(row.get('dose')):
                return np.nan
                
            unite = str(row['unite']).upper()
            if unite == 'KG/HA':
                return row['dose']
            elif unite == 'L/HA':
                return row['dose'] * (
                    self.organic_liquid_density 
                    if row['type_engrais'] == 'engrais organiques' 
                    else self.default_mineral_density
                )
            elif unite == 'M3/HA':
                return row['dose'] * (
                    self.organic_solid_density 
                    if row['type_engrais'] == 'engrais organiques'
                    else self.default_mineral_density * 1000
                )
            else:
                print(f"Avertissement: Unité non reconnue: {unite} pour l'engrais {row['libelle']}")
                return np.nan
        except Exception as e:
            print(f"Erreur lors de la conversion pour l'engrais {row['libelle']}: {str(e)}")
            return np.nan

    def _aggregate_by_exploitation(self, df, siret_column):
        """Agrège les données par exploitation avec pondération par surface"""
        print(f"Agrégation des données par exploitation ({siret_column})...")
        
        def weighted_average(group):
            # Calcul des moyennes pondérées par surface
            result = {}
            weights = group['surface_travaillee_ha']
            
            # Convertir les libellés en strings et supprimer les valeurs NaN
            libelles = group['libelle'].apply(lambda x: str(x) if pd.notnull(x) else '').unique()
            libelles = [lib for lib in libelles if lib != '']
            result['engrais_utilises'] = ', '.join(sorted(libelles, key=str))
            
            # Pour les engrais ICV
            if 'dose_kg_ha' in group.columns:
                icv_mask = group['in_icv_list']
                if icv_mask.any():
                    for fertilizer in group.loc[icv_mask, 'libelle'].unique():
                        if pd.notnull(fertilizer):  # Vérifier que le libellé n'est pas NaN
                            fertilizer_str = str(fertilizer)
                            mask = icv_mask & (group['libelle'].astype(str) == fertilizer_str)
                            if mask.any():
                                result[f"{fertilizer_str}_dose_kg_ha"] = np.average(
                                    group.loc[mask, 'dose_kg_ha'],
                                    weights=group.loc[mask, 'surface_travaillee_ha']
                                )
            
            # Pour les nutriments (non-ICV)
            non_icv_mask = ~group['in_icv_list']
            if non_icv_mask.any():
                # Ajouter la liste des engrais non-ICV séparément
                non_icv_libelles = group.loc[non_icv_mask, 'libelle'].apply(
                    lambda x: str(x) if pd.notnull(x) else '').unique()
                non_icv_libelles = [lib for lib in non_icv_libelles if lib != '']
                result['engrais_non_icv'] = ', '.join(sorted(non_icv_libelles, key=str))
                
                for nutrient in ['N_kg_ha', 'P_kg_ha', 'K_kg_ha']:
                    if nutrient in group.columns:
                        values = group.loc[non_icv_mask, nutrient]
                        weights_subset = group.loc[non_icv_mask, 'surface_travaillee_ha']
                        if not values.empty and not weights_subset.empty:
                            result[f"non_icv_{nutrient}"] = np.average(
                                values,
                                weights=weights_subset
                            )
            
            return pd.Series(result)
        
        # S'assurer que la colonne 'libelle' est de type string
        df['libelle'] = df['libelle'].astype(str)
        
        # Grouper par SIRET et agréger
        aggregated = df.groupby(siret_column).apply(weighted_average).reset_index()
        print(f"Données agrégées par exploitation : {len(aggregated)} exploitations")
        return aggregated

        # Grouper par SIRET et agréger
        aggregated = df.groupby(siret_column).apply(weighted_average).reset_index()
        print(f"Données agrégées par exploitation : {len(aggregated)} exploitations")
        return aggregated

    def combine_sources(self, mesparcelles_data, smag_data):
        """Combine les données des deux sources"""
        print("Combinaison des données...")
        
        # Standardisation des données
        print("\nTraitement des données MesParcelles...")
        mp_std = self.standardize_mesparcelles(mesparcelles_data)
        print("\nTraitement des données SMAG...")
        smag_std = self.standardize_smag(smag_data)
        
        # Ajout de la source
        mp_std['source'] = 'mesparcelles'
        smag_std['source'] = 'smag'
        
        # Vérification des colonnes
        print("\nColonnes dans les données standardisées:")
        print("MesParcelles:", mp_std.columns.tolist())
        print("SMAG:", smag_std.columns.tolist())
        
        # Combinaison des données
        combined = pd.concat([mp_std, smag_std], ignore_index=True)
        
        # Vérification des résultats
        print("\nRésumé des données combinées:")
        print(f"Nombre total d'observations: {len(combined)}")
        print(f"Nombre d'exploitations uniques: {combined['siret_exploitation'].nunique()}")
        print(f"Nombre de cultures uniques: {combined['culture'].nunique()}")
        print("\nListe des cultures uniques:")
        print(combined['culture'].unique())
        print("\nNombre d'observations par source et culture:")
        print(combined.groupby(['source', 'culture']).size())
        
        return combined

# Utilisation
if __name__ == "__main__":
    print("Chargement des données...")
    
    # Import des données
    mp_path = r"C:\Users\MonirNajem\OneDrive - FOOD PILOT\Desktop\monir\MESPARCELLES\mesparcelles_data.xlsx"
    smag_path = r"C:\Users\MonirNajem\OneDrive - FOOD PILOT\Desktop\monir\MESPARCELLES\SMAG.xlsx"
    
    mp_data = pd.read_excel(mp_path, sheet_name='Intervention_Details')
    smag_data = pd.read_excel(smag_path)
    
    # Standardisation et combinaison
    standardizer = FertilizerStandardizer()
    combined_data = standardizer.combine_sources(mp_data, smag_data)
    
    # Export des résultats
    combined_data.to_excel('resultats_standardisation_par_exploitation.xlsx', index=False)
    print("\nRésultats exportés dans 'resultats_standardisation_par_exploitation.xlsx'")

Chargement des données...
Combinaison des données...

Traitement des données MesParcelles...
Standardisation des données MesParcelles...
Après filtrage : 8 lignes
Agrégation des données par exploitation (siret_exploitation)...
Données agrégées par exploitation : 1 exploitations

Traitement des données SMAG...
Standardisation des données SMAG...
Agrégation des données par exploitation (siret_exploitation)...


C:\Users\MonirNajem\AppData\Local\Temp\ipykernel_28896\564241515.py:228: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated = df.groupby(siret_column).apply(weighted_average).reset_index()


Données agrégées par exploitation : 201 exploitations

Colonnes dans les données standardisées:
MesParcelles: ['siret_exploitation', 'engrais_utilises', 'Solution Liquide N 39_dose_kg_ha', 'engrais_non_icv', 'non_icv_N_kg_ha', 'non_icv_P_kg_ha', 'non_icv_K_kg_ha', 'source']
SMAG: ['siret_exploitation', 'engrais_utilises', 'engrais_non_icv', 'non_icv_N_kg_ha', 'non_icv_P_kg_ha', 'non_icv_K_kg_ha', 'source']

Résumé des données combinées:
Nombre total d'observations: 202
Nombre d'exploitations uniques: 202


C:\Users\MonirNajem\AppData\Local\Temp\ipykernel_28896\564241515.py:228: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated = df.groupby(siret_column).apply(weighted_average).reset_index()


KeyError: 'culture'